In [2]:
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import time
import datetime
import requests

In [3]:
URL = 'https://na.leagueoflegends.com/ko-kr/'

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(executable_path='../chromedriver', options=options)
driver.get(url=URL)

In [4]:
# 로그인 화면 들어가기
try:
    login_button = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH , '//*[@id="riotbar-right-content"]/div[3]/div[1]/a'))
    )
except:
    driver.quit()
login_button.click()

In [5]:
# username, password 넣기
try:
    username = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.NAME , 'username'))
    )
    password = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.NAME , 'password'))
    )
except:
    driver.quit()

In [6]:
with open("idpw.txt", 'r') as idpw:
  id, pw = idpw.read().split()
  username.send_keys(id)
  password.send_keys(pw)
  password.send_keys(Keys.RETURN)

In [7]:
time.sleep(5)
driver.get("https://matchhistory.kr.leagueoflegends.com/")
time.sleep(5)

In [8]:
# userId 받아오는 부분
userId = int(driver.current_url.split('/')[-1])
userId

203449844

In [9]:
s = requests.Session()
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
s.headers.update(headers)

for cookie in driver.get_cookies():
  c = {cookie['name'] : cookie['value']}
  s.cookies.update(c)

In [10]:
import json
import pprint

In [15]:
start = 0
end = 20
gameIds = []
gamerName = ""
while(start != end):
  match_info_get_url = f'https://acs.leagueoflegends.com/v1/stats/player_history/KR/{userId}?begIndex={start}&endIndex={end}&queue=0'
  res = s.get(match_info_get_url)
  # with open(f"{start}_{end}.log", "w") as log_file:
  #   pprint.pprint(json.loads(res.text), log_file)

  result = json.loads(res.text)
  if start == 0:
    gamerName = result['games']['games'][0]['participantIdentities'][0]['player']['summonerName']
    with open(f"../data/{gamerName}_userId.txt", "w") as userId_file:
      userId_file.write(str(userId))
  for game in result['games']['games']:
    if game['gameMode'] == 'CLASSIC':
      gameIds.append(game['gameId'])
  start = end
  if end < result['games']['gameCount']:
    end += 20

In [16]:
timeline = []
gameInfo = []
for gameId in gameIds:
  timeline_get_url = f"https://acs.leagueoflegends.com/v1/stats/game/KR/{gameId}/timeline"
  gameInfo_get_url = f"https://acs.leagueoflegends.com/v1/stats/game/KR/{gameId}"
  timeline_response = s.get(timeline_get_url)
  timeline.append(json.loads(timeline_response.text))
  gameInfo_response = s.get(gameInfo_get_url)
  gameInfo.append(json.loads(gameInfo_response.text))

In [20]:
with open(f"../data/{gameName}_timeline.json", "w") as json_file:
  json.dump(timeline, json_file)
with open(f"../data/{gameName}_gameInfo.json", "w") as json_file:
  json.dump(gameInfo, json_file)